# Access & Process 911 Calls

In [ ]:
# ACCESS CALLS

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install Kaggle
!pip install kaggle

# Copy kaggle.json from Google Drive
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/SmartEmergencyAssistant/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

# Verify Kaggle setup
!kaggle datasets list

# Download dataset from Kaggle
!kaggle datasets download -d louisteitelbaum/911-recordings-first-6-seconds -p /content/dataset

# Unzip the dataset
import zipfile
import os

zip_path = '/content/dataset/911-recordings-first-6-seconds.zip'
extract_path = '/content/dataset/911_calls'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(extract_path)

print("Files in dataset directory:", os.listdir(extract_path))

Mounted at /content/drive
ref                                                               title                                                size  lastUpdated                 downloadCount  voteCount  usabilityRating  
----------------------------------------------------------------  ---------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
atharvasoundankar/chocolate-sales                                 Chocolate Sales Data 📊🍫                             14473  2025-03-19 03:51:40.270000          13925        245  1.0              
adilshamim8/student-depression-dataset                            Student Depression Dataset                         467020  2025-03-13 03:12:30.423000           5695         93  1.0              
abdulmalik1518/mobiles-dataset-2025                               Mobiles Dataset (2025)                              20314  2025-02-18 06:50:24.370000          18937        310  1.0    

In [ ]:
# PROCESS CALLS + TRIM FILES

!pip install openai-whisper torch torchaudio

import whisper
import torchaudio
import os
import json
from tqdm import tqdm

# Define directory with audio files
audio_dir = '/content/dataset/911_calls/911_first6sec'
trimmed_dir = '/content/dataset/911_calls/trimmed_audio'

# Create directory for trimmed files
os.makedirs(trimmed_dir, exist_ok=True)

# Trim audio files to 60 seconds and save
for file_name in tqdm(os.listdir(audio_dir)):
  if file_name.endswith(".wav"):
    file_path = os.path.join(audio_dir, file_name)
    trimmed_path = os.path.join(trimmed_dir, file_name)

    try:
      waveform, sample_rate = torchaudio.load(file_path)
      trimmed_waveform = waveform[:, :sample_rate * 60] # Trim to 60 sec

      # Save trimmed file
      torchaudio.save(trimmed_path, trimmed_waveform, sample_rate)
    except Exception as e:
      print(f"Error processing {file_name}: {e}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 30.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.2 MB

100%|██████████| 708/708 [00:16<00:00, 43.15it/s]


In [ ]:
# GENERATE TRANSCRIPTIONS

# Load Whisper model
model = whisper.load_model("medium")

# Store transcriptions
transcriptions = {}

# Process each trimmed audio file
for file_name in tqdm(os.listdir(trimmed_dir)):
  if file_name.endswith(".wav"):
    file_path = os.path.join(trimmed_dir, file_name)

    try:
      # Generate transcription
      result = model.transcribe(file_path)
      transcriptions[file_name] = result["text"]
    except Exception as e:
      print(f"Error transcribing {file_name}: {e}")

transcription_path = '/content/drive/MyDrive/SmartEmergencyAssistant/transcriptions.json'

# Save all transcriptions to JSON file
with open (transcription_path, "w") as f:
  json.dump(transcriptions, f, indent=4)

print(f"Transcriptions completed for {len(transcriptions)} files.")



100%|█████████████████████████████████████| 1.42G/1.42G [01:35<00:00, 16.0MiB/s]
100%|██████████| 707/707 [18:47<00:00,  1.59s/it]

Transcriptions completed for 707 files.


In [ ]:
# FILTER RELEVANT CALLS BY KEYWORDS

import json

# Load transcriptions
with open(output_path, "r") as f:
    transcriptions = json.load(f)

# Expanded Keywords List
keywords = [
    "help", "emergency", "accident", "fire", "ambulance", "police", "medical", "rescue",
    "injury", "injured", "heart attack", "stroke", "bleeding", "unconscious", "choking",
    "seizure", "burn", "broken bone", "asthma", "overdose", "poison", "kidnapping", "assault",
    "robbery", "gunshot", "stabbed", "suicide", "drowning", "smoke", "explosion",
    "hazardous", "chemical", "flood", "earthquake", "tornado", "hurricane", "firefighter",
    "paramedic", "crime", "murder", "violence", "domestic", "abuse", "weapon", "shooting",
    "attack", "rape", "harassment", "disaster", "natural disaster", "urgent", "panic",
    "danger", "evacuate", "safety", "security", "trauma", "collapse", "distress", "fall",
    "hit", "crash", "collision", "wreck", "fatal", "hostage", "terrorism", "bomb",
    "biohazard", "missing", "child", "elderly", "pregnant", "severe", "critical", "dead",
    "fatality", "respiratory", "difficulty breathing", "chest pain", "shock", "amputation",
    "blood", "CPR", "resuscitation", "rescue operation", "recovery", "search and rescue",
    "missing person", "trapped", "hazard", "threat", "immediate", "response", "urgent care",
    "first aid", "lifesaving", "emergency service", "dangerous", "violence", "hostile",
    "shootout", "riot", "arson", "vandalism", "invasion", "break-in", "trespassing", "drug",
    "intoxicated", "drunk", "unresponsive", "shock", "allergic reaction", "anaphylaxis",
    "epilepsy", "dizzy", "fainting", "heatstroke", "hypothermia", "electrocution", "rescue",
    "aid", "disoriented", "disturbed", "chaos", "mass casualty", "hostile environment",
    "screaming", "yelling", "crying", "pleading", "begging", "911", "urgent request"
]

# Store relevant transcriptions
filtered_transcriptions = {}

for file_name, text in transcriptions.items():
    if any(keyword in text.lower() for keyword in keywords):
        filtered_transcriptions[file_name] = text

filtered_path = '/content/drive/MyDrive/SmartEmergencyAssistant/filtered_transcriptions.json'
# Save filtered transcriptions to a file
with open(filtered_path, "w") as f:
    json.dump(filtered_transcriptions, f, indent=4)

print(f"Filtered {len(filtered_transcriptions)} relevant clips out of {len(transcriptions)} total clips.")


Filtered 304 relevant clips out of 707 total clips.


# First Aid Intents Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install kaggle

!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/SmartEmergencyAssistant/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets list

!kaggle datasets download -d mahmoudahmed6/first-aid-intents-dataset -p /content/dataset

import zipfile
import os

zip_path = '/content/dataset/first-aid-intents-dataset.zip'
extract_path = '/content/dataset/first_aid_intents'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Files in dataset directory:", os.listdir(extract_path))

# PREPROCESSING THE DATASET

import json
import pandas as pd

# Load the dataset (Adjust this name if it's different)
with open('/content/dataset/first_aid_intents/intents.json', 'r') as file:
    intents_data = json.load(file)

# Extract list of intents from the dataset
intents = intents_data["intents"]

# Prepare lists to hold processed data
tags = []
patterns = []
responses = []

# Process each intent
for intent in intents:
  tag = intent["tag"].lower().strip()
  response = ' '.join(intent["responses"]).lower().strip()
  for pattern in intent["patterns"]:
    pattern_text = pattern.lower().strip()
    tags.append(tag)
    patterns.append(pattern_text)
    responses.append(response)

# Create DataFrame to organize preprocessed data
df = pd.DataFrame({
    "Tag": tags,
    "Pattern": patterns,
    "Response": responses
})

# Save the processed data to Google Drive
intents_path = '/content/drive/MyDrive/SmartEmergencyAssistant/preprocessed_first_aid_intents.csv'
df.to_csv(intents_path, index=False)


# Display the first few rows to confirm preprocessing
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ref                                                               title                                                size  lastUpdated                 downloadCount  voteCount  usabilityRating  
----------------------------------------------------------------  ---------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
atharvasoundankar/chocolate-sales                                 Chocolate Sales Data 📊🍫                             14473  2025-03-19 03:51:40.270000          13930        245  1.0              
adilshamim8/student-depression-dataset                            Student Depression Dataset                         467020  2025-03-13 03:12:30.423000           5700         93  1.0              
abdulmalik1518/mobiles-dataset-2025                               Mobiles Dataset (

,Tag,Pattern,Response
0,cuts,what to do if cuts?,wash the cut properly to prevent infection and...
1,cuts,how to cure cuts?,wash the cut properly to prevent infection and...
2,cuts,which medicine to apply for cuts?,wash the cut properly to prevent infection and...
3,cuts,what to apply on cuts?,wash the cut properly to prevent infection and...
4,cuts,cuts,wash the cut properly to prevent infection and...


# United-Syn-Med Preprocessing

In [6]:
# EXTRACT HF DATASET AND SAVE CSV

from google.colab import drive
drive.mount('/content/drive')

!pip install datasets
!pip install fuzzywuzzy[speedup] # For loose keyword filtering

from datasets import load_dataset
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Authenticating Hugging Face Login
from huggingface_hub import login
from google.colab import output
token = output.eval_js("prompt('Enter your HF Token: ')")
login(token=token)

# Download the Hugging Face Dataset
dataset = load_dataset("united-we-care/United-Syn-Med", split='train')

# Convert the dataset to a pandas DataFrame for inspection
df = pd.DataFrame(dataset)

# Inspect the dataset (prints first few rows)
print(df.head())

# Preprocessing (Converting to lowercase)
df = df.applymap(lambda s: s.lower() if type(s) == str else s)

# Define keywords to search for relevant entries
keywords = [
    "relief", "relieve", "treat", "treatment", "treating", "cure", "cured", "healing", "heal", "help", "remedy",
    "control", "prevent", "prevention", "aid", "first aid", "emergency", "response", "procedure", "therapy",
    "medicine", "medication", "drug", "symptoms", "diagnosis", "diagnose", "assessment", "solution", "assist",
    "recovery", "bandage", "compress", "disinfect", "clean", "wash", "pressure", "pain", "bleeding", "choking",
    "burn", "cut", "injury", "wound", "fracture", "infection", "poison", "breathe", "breathing", "drowning",
    "rescue", "restore", "stabilize", "emergency procedure", "bandaging", "compressing", "soothing", "application",
    "applying", "protecting", "medicating", "resuscitation", "CPR", "intubation", "antibiotic", "antiseptic",
    "ointment", "band-aid", "plaster", "wrap", "splint", "ice", "heat", "rehabilitation", "calm", "safety"
]

# Filter rows where the transcription column contains relevant keywords
def is_relevant(text):
    for keyword in keywords:
        if fuzz.partial_ratio(keyword, text) >= 85:  # Adjust threshold as needed (0-100)
            return True
    return False

filtered_df = df[df['transcription'].apply(lambda x: is_relevant(str(x)))]

# Save the filtered DataFrame to Google Drive
output_path = '/content/drive/MyDrive/SmartEmergencyAssistant/preprocessed_united_syn_med_filtered.csv'
filtered_df.to_csv(output_path, index=False)

# Confirm successful saving
print(f"Filtered dataset saved to: {output_path}")
print(f"Original dataset size: {len(df)} rows")
print(f"Filtered dataset size: {len(filtered_df)} rows")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                           file_name  \
0  drug-female-defa7fcb-89d7-4b25-8834-90888b201d...   
1  drug-female-160727b4-dd0c-43c7-ba17-963ae54347...   
2  drug-female-637d7dcc-fe73-499c-af76-b2ee28d363...   
3  drug-male-02a2daf6-0f99-4939-848d-adc95f03d4bd...   
4  drug-brand-en-us-male-421229aa-4f71-48fa-bd43-...   

                                       transcription  
0  Durysta is a medication used to reduce eye pre...  
1  Annona muricata extract is known for its poten...  
2  Many patients have found relief with REDBURY G...  
3  ALMAL-Z is a popular medication used for treat...  
4   Norfazole may cause side effects such as naus...  


<ipython-input-6-a15c16050760>:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda s: s.lower() if type(s) == str else s)


Filtered dataset saved to: /content/drive/MyDrive/SmartEmergencyAssistant/preprocessed_united_syn_med_filtered.csv
Original dataset size: 632548 rows
Filtered dataset size: 465670 rows


# Merging Datasets

In [7]:
from google.colab import drive
import pandas as pd
import json

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

### LOAD 911 CALLS DATASET (JSON to DataFrame) ###
transcriptions_path = '/content/drive/MyDrive/SmartEmergencyAssistant/filtered_transcriptions.json'
with open(transcriptions_path, 'r') as file:
    transcriptions_data = json.load(file)

# Convert the dictionary to a DataFrame
transcriptions_df = pd.DataFrame(list(transcriptions_data.items()), columns=['Input', 'Response'])
transcriptions_df['Response'] = None  # No standardized response
transcriptions_df['Source'] = '911 Calls'

### LOAD FIRST AID INTENTS DATASET (CSV to DataFrame) ###
intents_path = '/content/drive/MyDrive/SmartEmergencyAssistant/preprocessed_first_aid_intents.csv'
intents_df = pd.read_csv(intents_path)

# Ensure column names are consistent and add Source label
intents_df.rename(columns={'Pattern': 'Input'}, inplace=True)
intents_df['Source'] = 'First Aid Intents'

### LOAD HUGGING FACE DATASET (United-Syn-Med, CSV to DataFrame) ###
hf_path = '/content/drive/MyDrive/SmartEmergencyAssistant/preprocessed_united_syn_med_filtered.csv'
hf_df = pd.read_csv(hf_path)

# Rename columns appropriately
hf_df.rename(columns={'transcription': 'Input'}, inplace=True)
hf_df['Response'] = None  # No standardized response for the HF dataset
hf_df['Source'] = 'Medical Terminology'

### ENSURE COLUMN CONSISTENCY ACROSS ALL DATAFRAMES ###
intents_df = intents_df[['Input', 'Response', 'Source']]
hf_df = hf_df[['Input', 'Response', 'Source']]
transcriptions_df = transcriptions_df[['Input', 'Response', 'Source']]

### COMBINE ALL DATASETS INTO A SINGLE DATAFRAME ###
combined_df = pd.concat([intents_df, hf_df, transcriptions_df], ignore_index=True)

### SAVE COMBINED DATAFRAME TO GOOGLE DRIVE ###
combined_path = '/content/drive/MyDrive/SmartEmergencyAssistant/combined_dataset.csv'
combined_df.to_csv(combined_path, index=False)

### DISPLAY FIRST FEW ROWS TO CONFIRM SUCCESS ###
combined_df.head()


Mounted at /content/drive


,Input,Response,Source
0,what to do if cuts?,wash the cut properly to prevent infection and...,First Aid Intents
1,how to cure cuts?,wash the cut properly to prevent infection and...,First Aid Intents
2,which medicine to apply for cuts?,wash the cut properly to prevent infection and...,First Aid Intents
3,what to apply on cuts?,wash the cut properly to prevent infection and...,First Aid Intents
4,cuts,wash the cut properly to prevent infection and...,First Aid Intents


# Unsupervised Training (911 Calls + HF)

In [8]:
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
!pip install datasets
from datasets import Dataset
import torch
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Load the combined dataset from Google Drive
combined_path = '/content/drive/MyDrive/SmartEmergencyAssistant/combined_dataset.csv'
combined_df = pd.read_csv(combined_path)

# Filter out only the unlabeled data (where Response is None)
unlabeled_data = combined_df[combined_df['Response'].isna()]

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Prepare the text data for pretraining
texts = unlabeled_data['Input'].tolist()
tokenized_inputs = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

# Convert data to Hugging Face Dataset format
dataset = Dataset.from_dict({"input_ids": tokenized_inputs['input_ids'], "attention_mask": tokenized_inputs['attention_mask']})

# Data collator for MLM
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./bert_pretraining",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    report_to="none"
)

# Initialize the trainer
import os

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# Train the model
trainer.train()

# Save the pretrained model to Google Drive
pretrained_model_path = '/content/drive/MyDrive/SmartEmergencyAssistant/pretrained_model'
model.save_pretrained(pretrained_model_path)
tokenizer.save_pretrained(pretrained_model_path)

print(f"Pretrained model saved to: {pretrained_model_path}")


Mounted at /content/drive


<ipython-input-8-bfe5fa2bd47b>:13: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv(combined_path)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Step,Training Loss
100,2.353500
200,2.146600
300,2.003800
400,1.878200
500,1.851400
600,1.746300
700,1.757700
800,1.697600
900,1.714200
1000,1.659600


Pretrained model saved to: /content/drive/MyDrive/SmartEmergencyAssistant/pretrained_model


# Supervised Fine-Tuning (Intents)

In [3]:
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
!pip install datasets
from datasets import Dataset
import torch
import pandas as pd

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the preprocessed Intents dataset from Google Drive
intents_path = '/content/drive/MyDrive/SmartEmergencyAssistant/preprocessed_first_aid_intents.csv'
intents_df = pd.read_csv(intents_path)

# Filter out rows with empty responses to only keep usable data
intents_df = intents_df.dropna(subset=['Response'])

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('/content/drive/MyDrive/SmartEmergencyAssistant/pretrained_model')  # Your pre-trained model path

# Prepare the text data for fine-tuning
texts = intents_df['Pattern'].tolist()
responses = intents_df['Response'].tolist()

# Tokenize input and response pairs
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
labels = tokenizer(responses, truncation=True, padding=True, max_length=128, return_tensors='pt')['input_ids']

# Convert data to Hugging Face Dataset format
dataset = Dataset.from_dict({"input_ids": encodings['input_ids'], "attention_mask": encodings['attention_mask'], "labels": labels})

# Data collator for MLM
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./bert_finetuning",
    overwrite_output_dir=True,
    num_train_epochs=3,  # Increasing from 1 to 3 to improve fine-tuning quality
    per_device_train_batch_size=16,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=3e-5,
    weight_decay=0.01,
    report_to="none"  # Prevents logging to W&B; change this if you want logging
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# Train the model
trainer.train()

# Save the fine-tuned model to Google Drive
finetuned_model_path = '/content/drive/MyDrive/SmartEmergencyAssistant/finetuned_model'
model.save_pretrained(finetuned_model_path)
tokenizer.save_pretrained(finetuned_model_path)

print(f"Fine-tuned model saved to: {finetuned_model_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Step,Training Loss


Fine-tuned model saved to: /content/drive/MyDrive/SmartEmergencyAssistant/finetuned_model


# Upload Model to HF